In [8]:
import os
import pywt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [9]:
train_features_path = "../data/processed/balance_16k/preprocessing/x_train.csv"
labels_path = "../data/processed/balance_16k/preprocessing/y_train.csv"

In [10]:
def znorm(data):
    return (data - np.mean(data)) / (np.std(data))

def min_max(data):
    return (data- np.max(data))/ (np.max(data)- np.min(data))

In [4]:
df_train = pd.read_csv(train_features_path, header=None)
df_labels = pd.read_csv(labels_path, header=None)


In [5]:
def apply_cwt_to_single_row(row, num_columns_per_channel, scales, waveletname=""):
    BHZ = znorm(row[:num_columns_per_channel])
    BHN = znorm(row[num_columns_per_channel : 2 * num_columns_per_channel])
    BHE = znorm(row[2 * num_columns_per_channel :])

    cwt_BHZ, _ = pywt.cwt(BHZ, scales, waveletname)
    cwt_BHN, _ = pywt.cwt(BHN, scales, waveletname)
    cwt_BHE, _ = pywt.cwt(BHE, scales, waveletname)

    cwt_combined = np.stack([cwt_BHZ, cwt_BHN, cwt_BHE], axis=0)

    return cwt_combined


def apply_cwt_to_all_rows(data, num_columns_per_channel, scales, waveletname="morl"):
    cwt_results = []

    for _, row in data.iterrows():
        row_array = row.to_numpy()
        cwt_result = apply_cwt_to_single_row(
            row_array, num_columns_per_channel, scales, waveletname
        )
        cwt_results.append(cwt_result)

    return np.array(cwt_results)

In [6]:

df_combined = pd.concat([df_train, df_labels], axis=1)
columns = list(df_train.columns) + ['label']
df_combined.columns = columns
df_sampled = df_combined
df_train_sampled = df_sampled.iloc[:, :-1]  
df_labels_sampled = df_sampled.iloc[:, -1] 

In [ ]:
scales = np.geomspace(10, 2024, num=50)
num_columns_per_channel = 2401

cwt_transformed_data = apply_cwt_to_all_rows(df_train_sampled, num_columns_per_channel, scales)
def save_spectrogram_images_from_cwt(cwt_data, labels, output_folder):
    num_samples = cwt_data.shape[0]
    num_scales = cwt_data.shape[2]
    o = output_folder

    for i in range(num_samples):
        output_folder = o
        sample_data = cwt_data[i]
        label = labels[i]
        
        BHZ = np.array(sample_data[0])
        BHN = np.array(sample_data[1])
        BHE = np.array(sample_data[2]) 
        composite_image = np.zeros((num_scales, BHZ.shape[1], 3))
        composite_image[:, :, 0] = np.maximum(0,BHZ)
        composite_image[:, :, 1] = np.maximum(0,BHN)
        composite_image[:, :, 2] = np.maximum(0,BHE)

        if np.isnan(composite_image).any() or np.isinf(composite_image).any():
            print(f"Error: NaN or Inf values detected in composite_image for sample {i}. Skipping this sample.")
            continue

        max_value = composite_image.max()
        if max_value != 0:
            composite_image /= max_value
        else:
            print(f"Warning: Max value is zero for sample {i}. Skipping normalization.")
            continue        
        filename = f"{i}.png"
        if labels[i] == "Lg":
            output_folder += f"\{labels[i]}"
        elif labels[i] == "P":
            output_folder += f"\{labels[i]}"
        elif labels[i] == "Pg":
            output_folder += f"\{labels[i]}"
        elif labels[i] == "S":
            output_folder += f"\{labels[i]}"
        else:
            output_folder += f"\{labels[i]}"
        filepath = os.path.join(output_folder, filename)
        plt.figure(figsize=(10, 6))
        plt.imshow(composite_image, aspect='auto')
        plt.axis('off')    
        
        plt.savefig(filepath, format='png',bbox_inches='tight' ,pad_inches=0)
        plt.close() 

output_folder = "../spectrogram_images/"
save_spectrogram_images_from_cwt(cwt_transformed_data, df_labels_sampled, output_folder)